In [157]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange
import signatory

In [158]:
import torch
import torch.nn as nn

In [159]:
#device = "cuda" if torch.cuda.is_available() else "cpu"
#print("Using {} device".format(device))

data_type=torch.float32

In [160]:
x0 = 0# initial condition
sigma = 1 # volatility
segs=10
d=100
r =0 # risk free rate
batch_size = 100 # batch size
steps=200
T = 1 # maturity
dt = T/steps # mesh size
dt = T/steps # mesh size
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level

In [161]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [162]:
def create_stock(x0,r,sigma,T,steps,n_path,dW):  
    s_vec=[];
    w0=np.ones((n_path,1,d))*1e-6
    dW=np.concatenate((w0,dW),axis=1) ## the first time slot is then 0
    s_vec.append(np.ones((n_path,d))*x0)
    for i in range(steps): 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ dW[:,i+1,:]*sigma
        s_vec.append(s_vectemp)
    BM_path=np.cumsum(dW,axis=1) ## find the cumulative sum
    S_path=rearrange(np.array(s_vec), 'b c h -> c b h') 
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps, d=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt) ## This can be taken out
    times=np.append(times,T); 
    times_vec=np.tile(times,[1,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec=np.concatenate((times_vec, path),axis=2)
    return times_vec

def ComputeMultiLevelSig(s_embed, number_of_segment,level):
    n_batch, nsteps, chanels = s_embed.shape
    s_pathclass = signatory.Path(s_embed,level) ## Create a path class 

    t_vec = np.arange(0, nsteps-1, int(nsteps / number_of_segment))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []

    ll=torch.zeros(n_batch, 258) #iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
    MultiLevelSig.append(ll)
    for i in range(len(t_vec)-1):    
    ## Notice that we only use the signature of the concatenation of time and space.
        MultiLevelSig.append(s_pathclass.signature(0, t_vec[i+1]+1)) ##if not
        #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
    MultiLevelSig=torch.stack(MultiLevelSig)  
    MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    return MultiLevelSig

In [163]:
def generate_samples(model,batch_in=100):
    """
    Produce the signature, dW for computation, YT the terminal condition
    x_ten, selection the path and location to be used
    """
    dW=np.sqrt(dt)*np.random.normal(size=(batch_in, steps,d))
    
    pth2=create_stock(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    
    ########### embed the long sequence to a lower dimension
    BM_timePath=torch.tensor(BM_timePath,dtype=data_type)
    S_timePath=torch.tensor(S_timePath,dtype=data_type)
    s_embed=model.embedding(S_timePath)
    ###########
    
    sigs=ComputeMultiLevelSig(s_embed, segs, level)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1:]
    dW=BM_seg[:,1:,:]-BM_seg[:,:-1,:]

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)
    S_timePath=torch.tensor(S_timePath,dtype=torch.float32)
    
    x_ten=S_timePath[:,:,1:]
    
    YT=terminal_f(x_ten)
    
    return sigs, dW, YT, x_ten, selection[:-1]

In [164]:
def terminal_f(x_ten):
    res=(torch.sum(x_ten[:,:-1,:],dim=(1,2))*dt)**2
    return res

In [165]:
class Config(object):
    n_layer = 4
    
    dim=258; 
    Ntime=segs; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,1] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

### Determine the terminal loss

In [166]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.relu(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
    #    self.Y0=Parameter(torch.tensor([0.0])) #torch.rand(500,1)
        self.Y0=Parameter(torch.ones(100,1)*30) ## Has to do with the batch_size
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        self.embedding=signatory.Augment(in_channels=d+1,layer_sizes=(64,64,5),kernel_size=1,include_original=False,include_time=True) 
        
    def forward(self,batch_sig,batch_dW): 
   #     Z_path=self.model(batch_sig)
        Y=self.Y0
        for i in range(segs):
            Y=Y+sigma*torch.sum(self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:],axis=1,keepdim=True)
        return Y

## Check the definution of the final loss function

In [168]:
def Loss(Y_pred,Y_true):
    res=torch.mean((Y_pred-Y_true)**2)
    return res

In [169]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE=Lookback_PPDE(cfg)
optimizer=optim.Adam(model_PPDE.parameters(),lr=1e-2)

In [170]:
y0_mean=[];
loss_vec=[];
grad_clip=0.2

In [171]:
for i in range(5000):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(model_PPDE, batch_in=100)
    

    x_temp=model_PPDE(batch_sig,batch_dw)
    loss_temp=Loss(x_temp, batch_y)
    
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE.parameters(), grad_clip)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
    
    y0_val=model_PPDE.Y0.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)

    if i%10==0:
      #  print("Iter:", i,  loss_temp) 
        print("Iter:", i,  model_PPDE.Y0.mean(), loss_temp)

Iter: 0 tensor(30.0100, grad_fn=<MeanBackward0>) tensor(1841.9771, grad_fn=<MeanBackward0>)
Iter: 10 tensor(30.0756, grad_fn=<MeanBackward0>) tensor(2391.3469, grad_fn=<MeanBackward0>)
Iter: 20 tensor(30.1313, grad_fn=<MeanBackward0>) tensor(3909.6548, grad_fn=<MeanBackward0>)
Iter: 30 tensor(30.1989, grad_fn=<MeanBackward0>) tensor(1342.2900, grad_fn=<MeanBackward0>)
Iter: 40 tensor(30.2555, grad_fn=<MeanBackward0>) tensor(1887.8956, grad_fn=<MeanBackward0>)
Iter: 50 tensor(30.3312, grad_fn=<MeanBackward0>) tensor(2103.2075, grad_fn=<MeanBackward0>)
Iter: 60 tensor(30.4069, grad_fn=<MeanBackward0>) tensor(1075.0209, grad_fn=<MeanBackward0>)
Iter: 70 tensor(30.4699, grad_fn=<MeanBackward0>) tensor(5538.5063, grad_fn=<MeanBackward0>)
Iter: 80 tensor(30.5377, grad_fn=<MeanBackward0>) tensor(2066.1067, grad_fn=<MeanBackward0>)
Iter: 90 tensor(30.5925, grad_fn=<MeanBackward0>) tensor(1109.9388, grad_fn=<MeanBackward0>)
Iter: 100 tensor(30.6312, grad_fn=<MeanBackward0>) tensor(2517.7075, gr

Iter: 880 tensor(32.5908, grad_fn=<MeanBackward0>) tensor(2651.6699, grad_fn=<MeanBackward0>)
Iter: 890 tensor(32.5987, grad_fn=<MeanBackward0>) tensor(2906.7461, grad_fn=<MeanBackward0>)
Iter: 900 tensor(32.6013, grad_fn=<MeanBackward0>) tensor(1689.3262, grad_fn=<MeanBackward0>)
Iter: 910 tensor(32.5989, grad_fn=<MeanBackward0>) tensor(1706.1604, grad_fn=<MeanBackward0>)
Iter: 920 tensor(32.5939, grad_fn=<MeanBackward0>) tensor(1114.7058, grad_fn=<MeanBackward0>)
Iter: 930 tensor(32.5861, grad_fn=<MeanBackward0>) tensor(1717.7760, grad_fn=<MeanBackward0>)
Iter: 940 tensor(32.5939, grad_fn=<MeanBackward0>) tensor(4692.4082, grad_fn=<MeanBackward0>)
Iter: 950 tensor(32.6302, grad_fn=<MeanBackward0>) tensor(1332.4578, grad_fn=<MeanBackward0>)
Iter: 960 tensor(32.6368, grad_fn=<MeanBackward0>) tensor(3517.8940, grad_fn=<MeanBackward0>)
Iter: 970 tensor(32.6644, grad_fn=<MeanBackward0>) tensor(2077.2007, grad_fn=<MeanBackward0>)
Iter: 980 tensor(32.7094, grad_fn=<MeanBackward0>) tensor(10

Iter: 1750 tensor(33.0573, grad_fn=<MeanBackward0>) tensor(1908.9230, grad_fn=<MeanBackward0>)
Iter: 1760 tensor(33.1047, grad_fn=<MeanBackward0>) tensor(2582.0244, grad_fn=<MeanBackward0>)
Iter: 1770 tensor(33.1331, grad_fn=<MeanBackward0>) tensor(2519.7876, grad_fn=<MeanBackward0>)
Iter: 1780 tensor(33.1625, grad_fn=<MeanBackward0>) tensor(2968.8147, grad_fn=<MeanBackward0>)
Iter: 1790 tensor(33.2000, grad_fn=<MeanBackward0>) tensor(1333.2845, grad_fn=<MeanBackward0>)
Iter: 1800 tensor(33.2459, grad_fn=<MeanBackward0>) tensor(1932.1514, grad_fn=<MeanBackward0>)
Iter: 1810 tensor(33.2473, grad_fn=<MeanBackward0>) tensor(2899.5325, grad_fn=<MeanBackward0>)
Iter: 1820 tensor(33.2191, grad_fn=<MeanBackward0>) tensor(2245.7217, grad_fn=<MeanBackward0>)
Iter: 1830 tensor(33.1817, grad_fn=<MeanBackward0>) tensor(1606.8883, grad_fn=<MeanBackward0>)
Iter: 1840 tensor(33.1616, grad_fn=<MeanBackward0>) tensor(2001.2338, grad_fn=<MeanBackward0>)
Iter: 1850 tensor(33.1560, grad_fn=<MeanBackward0>

Iter: 2620 tensor(32.9668, grad_fn=<MeanBackward0>) tensor(2118.6287, grad_fn=<MeanBackward0>)
Iter: 2630 tensor(32.9415, grad_fn=<MeanBackward0>) tensor(2999.5806, grad_fn=<MeanBackward0>)
Iter: 2640 tensor(32.9316, grad_fn=<MeanBackward0>) tensor(2843.5747, grad_fn=<MeanBackward0>)
Iter: 2650 tensor(32.9103, grad_fn=<MeanBackward0>) tensor(3345.6235, grad_fn=<MeanBackward0>)
Iter: 2660 tensor(32.8850, grad_fn=<MeanBackward0>) tensor(1544.1320, grad_fn=<MeanBackward0>)
Iter: 2670 tensor(32.8548, grad_fn=<MeanBackward0>) tensor(1978.7710, grad_fn=<MeanBackward0>)
Iter: 2680 tensor(32.8440, grad_fn=<MeanBackward0>) tensor(3387.4875, grad_fn=<MeanBackward0>)
Iter: 2690 tensor(32.8428, grad_fn=<MeanBackward0>) tensor(3069.7031, grad_fn=<MeanBackward0>)
Iter: 2700 tensor(32.8265, grad_fn=<MeanBackward0>) tensor(1317.1624, grad_fn=<MeanBackward0>)
Iter: 2710 tensor(32.8091, grad_fn=<MeanBackward0>) tensor(2001.6172, grad_fn=<MeanBackward0>)
Iter: 2720 tensor(32.7798, grad_fn=<MeanBackward0>

Iter: 3490 tensor(33.0532, grad_fn=<MeanBackward0>) tensor(3003.4182, grad_fn=<MeanBackward0>)
Iter: 3500 tensor(33.0795, grad_fn=<MeanBackward0>) tensor(2830.9016, grad_fn=<MeanBackward0>)
Iter: 3510 tensor(33.0840, grad_fn=<MeanBackward0>) tensor(1870.4242, grad_fn=<MeanBackward0>)
Iter: 3520 tensor(33.0508, grad_fn=<MeanBackward0>) tensor(1819.2061, grad_fn=<MeanBackward0>)
Iter: 3530 tensor(33.0235, grad_fn=<MeanBackward0>) tensor(1045.6632, grad_fn=<MeanBackward0>)
Iter: 3540 tensor(33.0174, grad_fn=<MeanBackward0>) tensor(4568.3066, grad_fn=<MeanBackward0>)
Iter: 3550 tensor(33.0492, grad_fn=<MeanBackward0>) tensor(1735.8612, grad_fn=<MeanBackward0>)
Iter: 3560 tensor(33.0932, grad_fn=<MeanBackward0>) tensor(3120.5024, grad_fn=<MeanBackward0>)
Iter: 3570 tensor(33.1404, grad_fn=<MeanBackward0>) tensor(834.7338, grad_fn=<MeanBackward0>)
Iter: 3580 tensor(33.1227, grad_fn=<MeanBackward0>) tensor(2322.4792, grad_fn=<MeanBackward0>)
Iter: 3590 tensor(33.1173, grad_fn=<MeanBackward0>)

Iter: 4360 tensor(33.2227, grad_fn=<MeanBackward0>) tensor(1564.1143, grad_fn=<MeanBackward0>)
Iter: 4370 tensor(33.2174, grad_fn=<MeanBackward0>) tensor(1530.0164, grad_fn=<MeanBackward0>)
Iter: 4380 tensor(33.1989, grad_fn=<MeanBackward0>) tensor(3623.0544, grad_fn=<MeanBackward0>)
Iter: 4390 tensor(33.1581, grad_fn=<MeanBackward0>) tensor(2093.4883, grad_fn=<MeanBackward0>)
Iter: 4400 tensor(33.1129, grad_fn=<MeanBackward0>) tensor(2261.0696, grad_fn=<MeanBackward0>)
Iter: 4410 tensor(33.0832, grad_fn=<MeanBackward0>) tensor(3245.1401, grad_fn=<MeanBackward0>)
Iter: 4420 tensor(33.1043, grad_fn=<MeanBackward0>) tensor(1708.8776, grad_fn=<MeanBackward0>)
Iter: 4430 tensor(33.1052, grad_fn=<MeanBackward0>) tensor(2378.0398, grad_fn=<MeanBackward0>)
Iter: 4440 tensor(33.1274, grad_fn=<MeanBackward0>) tensor(1226.4042, grad_fn=<MeanBackward0>)
Iter: 4450 tensor(33.1248, grad_fn=<MeanBackward0>) tensor(1397.8174, grad_fn=<MeanBackward0>)
Iter: 4460 tensor(33.1302, grad_fn=<MeanBackward0>

In [172]:
y_pred=np.array(y0_mean)

In [175]:
np.abs(y_pred[4000:].mean()-33.33)/33.33

0.0053824107996736665

In [177]:
y_pred[4000:].mean()

33.150604

# dim=20, nlinear=10, step=100

In [121]:
for i in range(5000):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(model_PPDE, batch_in=100)
    

    x_temp=model_PPDE(batch_sig,batch_dw)
    loss_temp=Loss(x_temp, batch_y)
    
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE.parameters(), grad_clip)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
    
    y0_val=model_PPDE.Y0.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)

    if i%10==0:
      #  print("Iter:", i,  loss_temp) 
        print("Iter:", i,  model_PPDE.Y0.mean(), loss_temp)

Iter: 0 tensor(6.5004, grad_fn=<MeanBackward0>) tensor(59.3322, grad_fn=<MeanBackward0>)
Iter: 10 tensor(6.5034, grad_fn=<MeanBackward0>) tensor(170.8884, grad_fn=<MeanBackward0>)
Iter: 20 tensor(6.5053, grad_fn=<MeanBackward0>) tensor(91.3218, grad_fn=<MeanBackward0>)
Iter: 30 tensor(6.5066, grad_fn=<MeanBackward0>) tensor(60.0611, grad_fn=<MeanBackward0>)
Iter: 40 tensor(6.5075, grad_fn=<MeanBackward0>) tensor(95.5764, grad_fn=<MeanBackward0>)
Iter: 50 tensor(6.5076, grad_fn=<MeanBackward0>) tensor(42.5205, grad_fn=<MeanBackward0>)
Iter: 60 tensor(6.5068, grad_fn=<MeanBackward0>) tensor(108.7912, grad_fn=<MeanBackward0>)
Iter: 70 tensor(6.5074, grad_fn=<MeanBackward0>) tensor(98.8619, grad_fn=<MeanBackward0>)
Iter: 80 tensor(6.5084, grad_fn=<MeanBackward0>) tensor(80.6840, grad_fn=<MeanBackward0>)
Iter: 90 tensor(6.5084, grad_fn=<MeanBackward0>) tensor(43.1135, grad_fn=<MeanBackward0>)
Iter: 100 tensor(6.5096, grad_fn=<MeanBackward0>) tensor(83.3281, grad_fn=<MeanBackward0>)
Iter: 11

Iter: 900 tensor(6.5727, grad_fn=<MeanBackward0>) tensor(72.0325, grad_fn=<MeanBackward0>)
Iter: 910 tensor(6.5732, grad_fn=<MeanBackward0>) tensor(66.0637, grad_fn=<MeanBackward0>)
Iter: 920 tensor(6.5739, grad_fn=<MeanBackward0>) tensor(146.5367, grad_fn=<MeanBackward0>)
Iter: 930 tensor(6.5741, grad_fn=<MeanBackward0>) tensor(111.5719, grad_fn=<MeanBackward0>)
Iter: 940 tensor(6.5754, grad_fn=<MeanBackward0>) tensor(89.1382, grad_fn=<MeanBackward0>)
Iter: 950 tensor(6.5767, grad_fn=<MeanBackward0>) tensor(97.8874, grad_fn=<MeanBackward0>)
Iter: 960 tensor(6.5782, grad_fn=<MeanBackward0>) tensor(179.3801, grad_fn=<MeanBackward0>)
Iter: 970 tensor(6.5802, grad_fn=<MeanBackward0>) tensor(47.3338, grad_fn=<MeanBackward0>)
Iter: 980 tensor(6.5796, grad_fn=<MeanBackward0>) tensor(58.6446, grad_fn=<MeanBackward0>)
Iter: 990 tensor(6.5782, grad_fn=<MeanBackward0>) tensor(143.7928, grad_fn=<MeanBackward0>)
Iter: 1000 tensor(6.5778, grad_fn=<MeanBackward0>) tensor(51.6069, grad_fn=<MeanBackwa

Iter: 1790 tensor(6.5982, grad_fn=<MeanBackward0>) tensor(136.8042, grad_fn=<MeanBackward0>)
Iter: 1800 tensor(6.5962, grad_fn=<MeanBackward0>) tensor(124.3650, grad_fn=<MeanBackward0>)
Iter: 1810 tensor(6.5956, grad_fn=<MeanBackward0>) tensor(58.5300, grad_fn=<MeanBackward0>)
Iter: 1820 tensor(6.5947, grad_fn=<MeanBackward0>) tensor(54.1085, grad_fn=<MeanBackward0>)
Iter: 1830 tensor(6.5947, grad_fn=<MeanBackward0>) tensor(90.2083, grad_fn=<MeanBackward0>)
Iter: 1840 tensor(6.5953, grad_fn=<MeanBackward0>) tensor(64.8734, grad_fn=<MeanBackward0>)
Iter: 1850 tensor(6.5950, grad_fn=<MeanBackward0>) tensor(68.5335, grad_fn=<MeanBackward0>)
Iter: 1860 tensor(6.5946, grad_fn=<MeanBackward0>) tensor(62.9995, grad_fn=<MeanBackward0>)
Iter: 1870 tensor(6.5941, grad_fn=<MeanBackward0>) tensor(106.6329, grad_fn=<MeanBackward0>)
Iter: 1880 tensor(6.5935, grad_fn=<MeanBackward0>) tensor(165.5347, grad_fn=<MeanBackward0>)
Iter: 1890 tensor(6.5934, grad_fn=<MeanBackward0>) tensor(99.5990, grad_fn=<

Iter: 2680 tensor(6.6051, grad_fn=<MeanBackward0>) tensor(70.1861, grad_fn=<MeanBackward0>)
Iter: 2690 tensor(6.6053, grad_fn=<MeanBackward0>) tensor(117.3807, grad_fn=<MeanBackward0>)
Iter: 2700 tensor(6.6073, grad_fn=<MeanBackward0>) tensor(142.5843, grad_fn=<MeanBackward0>)
Iter: 2710 tensor(6.6097, grad_fn=<MeanBackward0>) tensor(93.8088, grad_fn=<MeanBackward0>)
Iter: 2720 tensor(6.6108, grad_fn=<MeanBackward0>) tensor(94.7923, grad_fn=<MeanBackward0>)
Iter: 2730 tensor(6.6113, grad_fn=<MeanBackward0>) tensor(103.1160, grad_fn=<MeanBackward0>)
Iter: 2740 tensor(6.6120, grad_fn=<MeanBackward0>) tensor(105.0852, grad_fn=<MeanBackward0>)
Iter: 2750 tensor(6.6135, grad_fn=<MeanBackward0>) tensor(82.3478, grad_fn=<MeanBackward0>)
Iter: 2760 tensor(6.6145, grad_fn=<MeanBackward0>) tensor(101.4899, grad_fn=<MeanBackward0>)
Iter: 2770 tensor(6.6163, grad_fn=<MeanBackward0>) tensor(76.5315, grad_fn=<MeanBackward0>)
Iter: 2780 tensor(6.6174, grad_fn=<MeanBackward0>) tensor(55.7483, grad_fn=

Iter: 3570 tensor(6.6280, grad_fn=<MeanBackward0>) tensor(93.3787, grad_fn=<MeanBackward0>)
Iter: 3580 tensor(6.6291, grad_fn=<MeanBackward0>) tensor(65.6053, grad_fn=<MeanBackward0>)
Iter: 3590 tensor(6.6298, grad_fn=<MeanBackward0>) tensor(121.4499, grad_fn=<MeanBackward0>)
Iter: 3600 tensor(6.6304, grad_fn=<MeanBackward0>) tensor(123.5354, grad_fn=<MeanBackward0>)
Iter: 3610 tensor(6.6332, grad_fn=<MeanBackward0>) tensor(103.7220, grad_fn=<MeanBackward0>)
Iter: 3620 tensor(6.6344, grad_fn=<MeanBackward0>) tensor(69.9853, grad_fn=<MeanBackward0>)
Iter: 3630 tensor(6.6356, grad_fn=<MeanBackward0>) tensor(55.2143, grad_fn=<MeanBackward0>)
Iter: 3640 tensor(6.6365, grad_fn=<MeanBackward0>) tensor(88.8436, grad_fn=<MeanBackward0>)
Iter: 3650 tensor(6.6376, grad_fn=<MeanBackward0>) tensor(134.5632, grad_fn=<MeanBackward0>)
Iter: 3660 tensor(6.6394, grad_fn=<MeanBackward0>) tensor(63.5638, grad_fn=<MeanBackward0>)
Iter: 3670 tensor(6.6392, grad_fn=<MeanBackward0>) tensor(49.2992, grad_fn=<

Iter: 4460 tensor(6.6237, grad_fn=<MeanBackward0>) tensor(100.3320, grad_fn=<MeanBackward0>)
Iter: 4470 tensor(6.6250, grad_fn=<MeanBackward0>) tensor(45.7016, grad_fn=<MeanBackward0>)
Iter: 4480 tensor(6.6258, grad_fn=<MeanBackward0>) tensor(85.2036, grad_fn=<MeanBackward0>)
Iter: 4490 tensor(6.6254, grad_fn=<MeanBackward0>) tensor(80.9854, grad_fn=<MeanBackward0>)
Iter: 4500 tensor(6.6258, grad_fn=<MeanBackward0>) tensor(78.1837, grad_fn=<MeanBackward0>)
Iter: 4510 tensor(6.6256, grad_fn=<MeanBackward0>) tensor(67.6079, grad_fn=<MeanBackward0>)
Iter: 4520 tensor(6.6264, grad_fn=<MeanBackward0>) tensor(98.1618, grad_fn=<MeanBackward0>)
Iter: 4530 tensor(6.6283, grad_fn=<MeanBackward0>) tensor(41.9579, grad_fn=<MeanBackward0>)
Iter: 4540 tensor(6.6278, grad_fn=<MeanBackward0>) tensor(113.1718, grad_fn=<MeanBackward0>)
Iter: 4550 tensor(6.6281, grad_fn=<MeanBackward0>) tensor(74.6195, grad_fn=<MeanBackward0>)
Iter: 4560 tensor(6.6276, grad_fn=<MeanBackward0>) tensor(198.3593, grad_fn=<M

In [176]:
(6.64-6.66)/6.66

-0.0030030030030030724

In [91]:
y_pred=np.array(y0_mean)
loss_var=np.array(loss_vec)
iters=np.arange(0,len(y_pred),1)+1

In [92]:
import pandas as pd

In [93]:
df=pd.DataFrame()
df['iter']=iters
df['y_pred']=y_pred
df['loss_var']=loss_var

In [95]:
df.to_csv('eg3_trainedMethod1_data//method1_1.csv')